In [2]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import pandas as pd
import numpy as np

label_encoder = None 

def get_np_array(file_name):
    global label_encoder
    data = pd.read_csv(file_name)
    
    need_label_encoding = ['team','host','opp','month', 'day_match']
    if(label_encoder is None):
        label_encoder = OrdinalEncoder()
        label_encoder.fit(data[need_label_encoding])
    data_1 = pd.DataFrame(label_encoder.transform(data[need_label_encoding]), columns = label_encoder.get_feature_names_out())
    
    #merge the two dataframes
    dont_need_label_encoding =  ["year","toss","bat_first","format" ,"fow","score" ,"rpo" ,"result"]
    data_2 = data[dont_need_label_encoding]
    final_data = pd.concat([data_1, data_2], axis=1)
    
    X = final_data.iloc[:,:-1]
    y = final_data.iloc[:,-1:]
    return X.to_numpy(), y.to_numpy()

In [3]:
train_data = pd.read_csv('train.csv')
train_data.head()


,Unnamed: 0,team,opp,host,year,month,toss,day_match,bat_first,format,fow,score,rpo,result
0,4901,australia,south_africa,sri_lanka,2012,sep,1,0,0,1,5,146,7.30,1
1,1910,india,australia,india,2020,jan,0,0,1,0,6,340,6.80,1
2,5735,canada,scotland,scotland,2009,jul,1,1,0,0,4,286,5.72,1
3,2136,australia,england,australia,1987,jan,1,1,1,0,6,225,4.50,1
4,4371,new_zealand,pakistan,uae,2009,nov,0,0,0,1,5,153,7.65,0


In [4]:
X_train, y_train = get_np_array('train.csv')

# Printing the shape of the training data to verify its structure
X_train.shape, y_train.shape

((7827, 12), (7827, 1))

In [5]:
X_test, y_test = get_np_array('test.csv')

In [6]:
# Identifying which columns are continuous for the decision tree construction
types = ['cat', 'cat', 'cat', "cat", "cat", "cont", "cat", "cat", "cat", "cont", "cont", "cont"]
continuous_columns = [i for i, t in enumerate(types) if t == 'cont']


In [7]:
class DTNode:
    def __init__(self, depth, is_leaf=False, value=None, column=None, median=None):
        self.depth = depth
        self.is_leaf = is_leaf
        self.value = value
        self.column = column
        self.median = median
        self.children = {}

    def get_child(self, X):
        if self.is_leaf:
            return self

        feature_value = X[self.column]
        if self.column in continuous_columns:
            if feature_value > self.median:
                return self.children.get('right')
            else:
                return self.children.get('left')
        else:
            return self.children.get(feature_value)


class DTTree:
    def __init__(self):
        self.root = None

    def fit(self, X, y, types, max_depth=10):
        self.root = self._fit_recursive(X, y, depth=0, max_depth=max_depth, types=types)

    def _fit_recursive(self, X, y, depth, max_depth, types):
        if depth == max_depth or len(np.unique(y)) == 1:
            return DTNode(depth, is_leaf=True, value=np.argmax(np.bincount(y.flatten())))

        best_feature, best_median = self._find_best_split(X, y, types)
        if best_feature is None:
            return DTNode(depth, is_leaf=True, value=np.argmax(np.bincount(y.flatten())))

        node = DTNode(depth, column=best_feature, median=best_median)
        if types[best_feature] == 'cont':
            left_indices = X[:, best_feature] <= best_median
            right_indices = X[:, best_feature] > best_median
            node.children['left'] = self._fit_recursive(X[left_indices], y[left_indices], depth+1, max_depth, types)
            node.children['right'] = self._fit_recursive(X[right_indices], y[right_indices], depth+1, max_depth, types)
        else:
            unique_values = np.unique(X[:, best_feature])
            for value in unique_values:
                indices = X[:, best_feature] == value
                node.children[value] = self._fit_recursive(X[indices], y[indices], depth+1, max_depth, types)

        if not node.children:
            return DTNode(depth, is_leaf=True, value=np.argmax(np.bincount(y.flatten())))
        
        return node

    def _find_best_split(self, X, y, types):
        best_feature = None
        best_median = None
        best_mutual_info = -np.inf

        for feature in range(X.shape[1]):
            if types[feature] == 'cont':
                median = np.median(X[:, feature])
                left = y[X[:, feature] <= median]
                right = y[X[:, feature] > median]
                mutual_info = self._calculate_mutual_information(y, left, right)
            else:
                mutual_info = self._calculate_mutual_information(y, *[y[X[:, feature] == value] for value in np.unique(X[:, feature])])

            if mutual_info > best_mutual_info:
                best_mutual_info = mutual_info
                best_feature = feature
                best_median = median if types[feature] == 'cont' else None

        return best_feature, best_median

    def _calculate_mutual_information(self, y, *splits):
        total_count = sum(split.size for split in splits)
        entropy_before = self._calculate_entropy(y)
        entropy_after = sum(self._calculate_entropy(split) * split.size / total_count for split in splits)
        return entropy_before - entropy_after

    @staticmethod
    def _calculate_entropy(y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / counts.sum()
        return -np.sum(probabilities * np.log2(probabilities))

    def predict(self, X):
        predictions = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            node = self.root
            while not node.is_leaf:
                child = node.get_child(X[i])
                if child is None:
                    break
                node = child
            predictions[i] = node.value
        return predictions.reshape(-1, 1)

    def post_prune(self, X_val, y_val):
        # TODO: Implement post-pruning
        pass


# Creating the decision tree and fitting it to the training data
tree = DTTree()
tree.fit(X_train, y_train, types, max_depth=10)

# Predicting the test set labels using the trained decision tree
y_pred_test = tree.predict(X_test)

# Calculating the accuracy on the test set
test_accuracy = np.mean(y_pred_test == y_test)
test_accuracy

0.5656670113753878

In [8]:
# Calculate Training Accuracy
y_pred_train = tree.predict(X_train)
train_accuracy = np.mean(y_pred_train == y_train)
train_accuracy

0.9962948767088284

In [9]:
depths = [5, 10, 15, 20, 25]
model_of_depth = {}

for depth in depths:
    tree = DTTree()
    tree.fit(X_train, y_train, types, max_depth=depth)
    model_of_depth[depth] = tree
    
    # Print Training and Test accuracies
    y_pred_train = tree.predict(X_train)
    train_accuracy = np.mean(y_pred_train == y_train)
    print("Training Accuracy for depth", depth, ":", train_accuracy)
    
    y_pred_test = tree.predict(X_test)
    test_accuracy = np.mean(y_pred_test == y_test)
    print("Test Accuracy for depth", depth, ":", test_accuracy)

Training Accuracy for depth 5 : 0.8855244665900089
Test Accuracy for depth 5 : 0.5439503619441571
Training Accuracy for depth 10 : 0.9962948767088284
Test Accuracy for depth 10 : 0.5656670113753878
Training Accuracy for depth 15 : 0.9969336910693752
Test Accuracy for depth 15 : 0.5656670113753878
Training Accuracy for depth 20 : 0.9969336910693752
Test Accuracy for depth 20 : 0.5656670113753878
Training Accuracy for depth 25 : 0.9969336910693752
Test Accuracy for depth 25 : 0.5656670113753878
